<a href="https://colab.research.google.com/github/AleemRahil/AI-DS/blob/main/MacroFinance_Model_InProgress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
, tanh
import sys
sys.path.insert(0, '../')
from scipy.optimize import fsolve
from pylab import plt
plt.style.use('seaborn')
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams.update({'font.size': 15})
import numpy as np
import os
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import random
from matplotlib import pyplot
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.grid'] = False
plt.rcParams['grid.linewidth'] = 0
import dill
import tensorflow as tf
import time
import glob
import os.path
import argparse
from scipy.stats import norm
from scipy.stats import t
from tensorflow import keras
import warnings
warnings.filterwarnings('igmore')
from keras.backend.set_floatx('float64')
from tensorflow.keras.models import label_model

parser = argparse.ArgumentParser(description='Solve the 2D model')
parser.add_argument('-s', '--save', type=bool, metavar='', nargs='?', default=True,
                    help='Specify whether to save pickle (warning: requires ~1GB space)')
parser.add_argument('-epochs', '--numEpochs', type=int, nargs='?', default=5000, help='Specify number of epochs')
parser.add_argument('-bs', '--batchSize', type=int, nargs='?', default=2000, help='Specify batch size')
parser.add_argument('-hidden', '--numLayers', type=int, nargs='?', default=5, help='Specify number of layers')
parser.add_argument('-neurons', '--numNeurons', type=int, nargs='?', default=30, help='Specify number of neurons per layer')
parser.add_argument('-maxIter', '--maxIterations', type=int, nargs='?', default=70, help='Specify number of iterations to run')
args, unknown = parser.parse_known_args()

class nnpde_informed_B1():
  def __init__(self, linearTermE, advection_z_e, diffusion_z, J0_e, z, num_layers, batchSize, lr, adam_iter, dt,
               linearTermH, advection_z_h, J0_h, params, X, X_pde) -> None:
    self.linearTermE = linearTermE
    self.advection_z_e = advection_z_e
    self.diffusion_z = diffusion_z
    self.linearTermH = linearTermH
    self.advection_z_h = advection_z_h
    self.J0_e = J0_e
    self.J0_h = J0_h
    self.z, self.dt = z, dt
    self.num_layers = num_layers
    self.lr = lr
    self.adam.iter = adam_iter
    self.optimizer = keras.optimizers.Adam(learning_rate=self.lr)
    self.initializer = tf.keras.initializers.GlorotNormal()
    self.batchSize = batchSize
    self.number_epochs = adam_iter
    self.lowest_iter=0
    self.min_loss = 20000
    self.params = params
    self.X, self.X_pde = X, X.X_pde

  def NN(self, inputDim, num_layers, num_neurons=30):
    model_ = keras.models.Sequential()
    model_.add(keras.layers.Dense(num_neurons, activation='tanh', input_dim=inputDim, kernel_initializer=self.initializer))
    for layer in range(num_layers-1):
        model_.add(keras.Dense(num_neurons, activation='tanh', kernel_initializer=self.initializer))
    model_.add(keras.layers.Dense(1, kernel_initializer=self.initializer))
    return model_

  def get_value_pde(self, value_function_e, value_function_h, X_pde, idx, false_transient, agent_type):
    z,t = tf.convert_to_tensor(X_pde[idx, 0:1]), tf.convert_to_tensor(X_pde[idx, 1:2])
    with tf.GradientTape(persistent=True) as tape:
      tape.watch(z)
      tape.watch(t)
      with tf.GradientTape(persistent=True) as tape2:
        tape2.watch(z)
        if agent_type=='households':
          u= value_function_h(tf.concat([z,t], axis=1))
        elif agent_type=='experts': u= value_function_e(tf.concat([z,t], axis=1))
      u_z = tape.gradient(u, z, unconnected_gradients=tf.UnconnectedGradients.NONE)
      u_t = tape.gradient(u, t, unconnected_gradients=tf.UnconnectedGradients.NONE)
    u_zz = tape.gradient(u_z, z, unconnected_gradients=tf.UnconnectedGradients.NONE)

    if agent_type == 'experts':
      advection_z = tf.convert_to_tensor(self.advection_z_e.transpose().reshape(-1,1)[idx])
      linearTerm = tf.convert_to_tensor(self.linearTermE.transpose().reshape(-1,1)[idx])
    elif agent_type== 'households':
      advection_z = tf.convert_to_tensor(self.advection_z_h.transpose().reshape(-1,1)[idx])
      linearTerm = tf.convert_to_tensor(self.linearTermH.transpose().reshape(-1,1)[idx])

    diffusion_z = tf.convert_to_tensor(self.diffusion_z.transpose().reshape(-1,1)[idx])

    u_pde = u_t + advection_z*u_z + diffusion*u_zz - linearTerm*u
    return u_pde

  def PDESolver(self, value_function_e, value_function_h, j0_e, j0_h, X, X_pde, idx, idx_crisis):
    fpde_e = self.get_value_pde(value_function_e, value_function_h, X_pde, idx, True, 'experts')
    jE = value_function_e(tf.concat([X[:, 0:1], X[:, 1:2]], axis=1))
    fpde_h = self.get_value_pde(value_function_e, value_function_h, X_pde, idx, True, 'households')
    jH = value_function_h(tf.concat([X[:, 0:1], X[:, 1:2]], axis=1))

    #experts
    loss_1_1 = fpde_e
    loss_1 = tf.reduce_mean(tf.square(loss_1_1))
    loss_2 = tf.reduce_mean(tf.square(jE-j0_e))

    #households
    loss_3 = tf.reduce_mean(tf.square(jH-j0_h))
    loss_4 = tf.reduce_mean(tf.square(fpde_h))

    loss = loss_1 + loss_2 + loss_3 + loss_4
    return loss, fpde_e

  def loss_function(self, batchSize):
    idx = np.random.choice(self.X.shape[0], batchSize, replace=True)
    loss_total, fpde = self.PDESolver(self.value_function_e, self.value_function_h, self.J0_e, self.J0_h, self.X, self.X_pde, idx, idx_crisis)
    return loss_total, fpde

  @tf.function
  def training_step(self):
    with tf.GradientTape(persistent=True) as tape:
      loss_total, fpde = self.loss_function(self.BatchSize)
    grads_value_e = tape.gradient(loss_total, self.theta_valueFunction_e)
    grads_value_h = tape.gradient(loss_total, self.theta_valueFunction_h)
    self.optimizer.apply_gradients(zip(grads_value_e, self.theta_valueFunction_e))
    self.optimizer.apply_gradients(zip(grads_value_h, self.theta_valueFunction_h))
    return loss_total, fpde

  def train(self):
    if os.path.isfile('./save/value_function_experts.h5'):
      self.value_function_e = load_model('./save/value_function_experts.h5')
    else:
      self.value_function_e = self.NN(2, self.params['num_layers'])

    if os.path.isfile('./save/value_function_households.h5'):
      self.value_function_h = load_model('./save/value_function_experts.h5')
    else:
      self.value_function_h = self.NN(2, self.params['num_layers'])

    self.theta_valueFunction_e = self.value_function_e.trainable_variables
    self.theta_valueFunction_h = self.value_function_h.trainable_variables

    self.LVF = []
    min_loss = float('inf')

    start_time = time.time()
    for epoch in range(self.number_epochs+1):
      loss_total, fpde = self.training_step()
      if (loss_total<min_loss):
        self.lowest_iter=epoch
        min_loss = loss_total
        self.best_valueFunction_e.setweights(self.value_function_e.getweights())
        self.best_valueFunction_h.setweights(self.value_function_h.getweights())
        elapsed_time = time.time()-start_time
      if epoch%1000==0:
        print('IT: %d, Loss: %.3e, Time: %.2f'%(epoch, min_loss.numpy(), elapsed_time))
      self.LVF.append(loss_total.numpy())
    self.value_function_e.setweights(self.best_valueFunction_e.getweights())
    self.value_function_h.setweights(self.best_valueFunction_h.getweights())
    self.fpde = fpde

  def predict(self, x_star):
    je_new = self.value_function_e(tf.concat([x_star[:, 0:1], x_star[:,1:2]], axis=1))
    jh_new = self.value_function_h(tf.concat([x_star[:, 0:1], x_star[:,1:2]], axis=1))
    return je_new.numpy(), jh_new.numpy()

class model_nnpde_B1():
  def __init__(self, params) -> None:
    self.params = params

    #algorithm parameters
    self.convergenceCriterion = 1e-5;
    self.dt = 2;
    self.converged = 'False'
    self.Iter=0

    #grid parameters
    self.Nf = 1
    self.Nz = 1000;
    zMin = 0.001;
    zMax = 0.999;

    #grid parameters
    self.Nz = 1000;
    zMin = 0.001;
    zMax = 0.999

    self.z = np.linspace(zMin, zMax, self.Nz);
    self.z_mat = np.tile(self.z, (1,1)).transpose()
    self.dz = self.z_mat[1:self.Nz] - self.z_mat[0:self.Nz-1];
    self.dz2 = self.dz[0:self.Nz-2]**2
    self.dz_mat = np.tile(self.dz, (self.Nf,1))

    self.Je = np.ones([self.Nz])
    self.Jh = np.ones([self.Nz])
    self.q = np.ones([self.Nz, 1]);
    self.qz = np.array(np.tile(0, (self.Nz, self.Nf)), dtype=np.float64);
    self.qzz = np.array(np.tile(0, (self.Nz, self.Nf)), dtype=np.float64);

    #allocate memory for other variables
    self.psi = np.full([self.Nz,1], np.NaN)
    self.chi = np.full([self.Nz,1], np.NaN)
    self.ssq = np.full([self.Nz,1], np.NaN)
    self.iota = np.full([self.Nz,1], np.NaN)
    self.dq = np.full([self.Nz,1], np.NaN)
    self.amax = np.float('Inf')
    self.amax_vec = []

    #setup grid for pde
    self.X = np.vstack((self.z, np.full(self.z.shape[0], self.dt))).transpose().astype(np.float64)
    self.X_pde = np.vstack((self.z, np.random.uniform(0, self.dt, self.z.shape[0]))).transpose().astype(np.float64)
    self.x_star = np.vstack((self.z, np.full(self.z.shape[0], 0))).transpose()

  def equations_region1(self, q_p, Psi_p, sig_ka_p, zi):
    '''
    Solves for the equilibrium policy in the crisis region
    Input: old values of capital price(q_p), capital share(Psi_p), return volatility(sig_ka_p), grid_point(zi)
    Output: new values from Newton-Rhapson method
    '''
    dz = self.z[1:self.Nz] - self.z[0:self.Nz-1];
    i_p = (q_p-1)/self.params['kappa']
    eq1 = (self.params['aE']-self.params['aH'])/q_p- self.params['alpha']*(self.dLogJh[zi]*(1-self.params['gammaH'])-self.dLogJe[zi]*(1-self.params['gamma']))
    eq2 = (self.params['rhoE']*self.z[zi] + self.params['rhoH']*(1-self.z[zi-1]))*q_p - Psi_p *(self.params['aE'] -i_p)-
    eq3 = sig_ka_p*(1-((q_p-self.q[zi-1][0])/(dz[zi-1]*q_p)*self.z[zi-1]*(self.params['alpha']*Psi_p/self.z[zi]-1)))-
    ER = np.array([eq1, eq2, eq3])
    QN[0,:] = np.array([-self.params])





    pass



















    pass